# Build custom databases with segment flu genomes using kraken_flu utility
A new utilty [kraken_flu](https://gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu) was created to generate Kraken2 databases with segmented flu genomes from any set of input files (either from kraken2 or directly from NCBI).

The tool peforms two tasks:
1. Fiter influenza genomes to keep only those that have 8 full-length segments
2. Create a reorganisee taxonomy with new taxa for segments of the influenza A viruses

This notebook builds a complete database with NCBI RefSeq plus the NCBI Influenza resource.

## directory paths and names

In [1]:
export DB_NAME=refseq_ncbiFlu_kfv2_102424
export BASE_DIR=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/

# taxonomy data is universal and ideally reused for every DB we build, so the direcotry is not specific to this DB
export TAX_PATH=${BASE_DIR}/downloads/taxonomy_download/

# the contents of this folder were copied from refseq_ncbiFlu_kfv2_091124
export LIB_PATH=${BASE_DIR}/downloads/sequence_download/refseq_ncbiFlu_kfv2/

export DB_PREP_DIR=${BASE_DIR}/db_prep/${DB_NAME}/
export DB_PATH=${BASE_DIR}/databases/${DB_NAME}

In [2]:
mkdir -p ${BASE_DIR}

rm -rf ${DB_PREP_DIR}
mkdir -p ${DB_PREP_DIR}
mkdir -p ${DB_PATH}

In [3]:
echo "BASE DIR: $BASE_DIR"
echo "TAXONOMY PATH: ${TAX_PATH}"
echo "LIBRARY (FASTA) PATH: ${LIB_PATH}"
echo "DB PREP DIR: ${DB_PREP_DIR}"
echo "FINAL DB PATH: ${DB_PATH}"

BASE DIR: /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu/
TAXONOMY PATH: /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//downloads/taxonomy_download/
LIBRARY (FASTA) PATH: /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//downloads/sequence_download/refseq_ncbiFlu_kfv2/
DB PREP DIR: /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//db_prep/refseq_ncbiFlu_kfv2_102424/
FINAL DB PATH: /lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//databases/refseq_ncbiFlu_kfv2_102424


## install the kraken_flu tool
The tool can be installed from local gitlab. Creating a venv for it.

In [4]:
python3 -m venv ~/kraken_flu

In [4]:
source ~/kraken_flu/bin/activate

In [5]:
pip install --upgrade pip

___This version of kraken-flu is not yet in main branch, checking out latest commit in dev branch___

In [5]:
COMMIT_SHA=4583004bd0ad86959178ac17d0fe1bd0648dab85

In [6]:
pip install kraken_flu@git+ssh://git@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git@$COMMIT_SHA

  Cloning ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git (to revision 4583004bd0ad86959178ac17d0fe1bd0648dab85) to /tmp/pip-install-sy8lagvm/kraken-flu_b95d2d5943e745ef8d48883ce9a70a11
  Running command git clone --filter=blob:none --quiet 'ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git' /tmp/pip-install-sy8lagvm/kraken-flu_b95d2d5943e745ef8d48883ce9a70a11
  Running command git rev-parse -q --verify 'sha^4583004bd0ad86959178ac17d0fe1bd0648dab85'
  Running command git fetch -q 'ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git' 4583004bd0ad86959178ac17d0fe1bd0648dab85
  Running command git checkout -q 4583004bd0ad86959178ac17d0fe1bd0648dab85
  Resolved ssh://****@gitlab.internal.sanger.ac.uk/malariagen1/misc_utils/kraken_flu.git to commit 4583004bd0ad86959178ac17d0fe1bd0648dab85
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.t

In [7]:
kraken_flu -v

kraken_flu 2.1.1.dev6+g4583004


## Download data
Get viral taxonomy and sequence data from NCBI

In [8]:
module load kraken2/2.1.2

	Module loaded. For more information run 'module help kraken2/2.1.2'.
	- By default, kraken2 will search for named databases in /data/pam/software/kraken2 and the current working directory. The full path to
	any database can also be used.
	- `kraken2-build` commands will sometimes experience an error suggesting that it is not possible to download a library using rsync. While the option --use-ftp
	may fix this, rsync tends to be faster and more reliable. Try re-running the `kraken2-build` command until the download is successful. Note that `kraken2-build`
	will produce checkpoints during the installation process and will restart at the last successful step.
	- If `kraken2-build build` freezes or hangs indefinitely, using the `--fast-build` option may help. An earlier version of the software (if available) may also
	allow the build step to proceed successfully.


### Taxonomy data
Use the kraken2 build tool to download taxonomy files from [NCBI](https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/).

___This only needs to be once, hence commented out here. Uncomment to re-run___

In [11]:
# kraken2-build --download-taxonomy --db ${TAX_PATH}

In [9]:
tree ${TAX_PATH}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//downloads/taxonomy_download/
└── taxonomy
    ├── accmap.dlflag
    ├── citations.dmp
    ├── delnodes.dmp
    ├── division.dmp
    ├── gc.prt
    ├── gencode.dmp
    ├── images.dmp
    ├── merged.dmp
    ├── names.dmp
    ├── nodes.dmp
    ├── nucl_gb.accession2taxid
    ├── nucl_wgs.accession2taxid
    ├── readme.txt
    ├── taxdump.dlflag
    ├── taxdump.tar.gz
    └── taxdump.untarflag

1 directory, 16 files


### Sequence data
Directly download from NCBI RefSeq release FTP website (file from 15/01/24

NCBI viral RefSeq (directly from NCBI now, not using the kraken2 pre-built)

___This step is not required to run every time and it takes a long time, hence commented out___

In [18]:
cd ${LIB_PATH}
#wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.1.genomic.fna.gz
#gunzip viral.1.1.genomic.fna.gz

NCBI Infuenza FTP  
___NOTE___ that the resource is no longer kept updated (since Oct 2020) so this is used as a starting point but we still need to add later influenza genomes using the new NCBI API for

https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/

Here is a post showing how to programmatically interact with the new NCBI Virus site to obtain sequences:  
https://www.biostars.org/p/9562294/   

___This step is not required to run every time and it takes a long time, hence commented out___

In [14]:
cd ${LIB_PATH}
# wget https://ftp.ncbi.nih.gov/genomes/INFLUENZA/influenza.fna

In [20]:
tree ${LIB_PATH}

/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//downloads/sequence_download/refseq_ncbiFlu_kfv2/
├── influenza.fna
├── sequences_RSV_A.fasta
├── sequences_RSV_B.fasta
└── viral.1.1.genomic.fna

0 directories, 4 files


___RSV sequences___ were downloaded from https://nextstrain.org/rsv/a/genome/6y?d=tree,entropy&p=full

## Run the kraken-flu tool
The tool creates a new directory of taxonomy and sequence files.  

Using an exception to the "complete flu genomes" filter for the avian flu reference because we want this one in the DB but it does not have all 8 segments in RefSeq.

___NOTE___ The current implementation of the kraken-flu tool reads sequence and taxonomy files into RAM and therefore needs a significant amount of RAM. The process got killed on a 5GB node but work fine on 10GB.


In [10]:
rm ${DB_PREP_DIR}/kfdb.sqlite
~/kraken_flu/bin/kraken_flu \
    --taxonomy_path  ${TAX_PATH}/taxonomy \
    --fasta_path ${LIB_PATH}/influenza.fna ${LIB_PATH}/viral.1.1.genomic.fna \
    --out_dir ${DB_PREP_DIR} \
    --db_file ${DB_PREP_DIR}/kfdb.sqlite \
    --keep_db_file \
    --filter_flu \
    --filter_except "A/Goose/Guangdong/1/96(H5N1)" \
    --do_full_linkage \
    --rsv_a_sequences ${LIB_PATH}/sequences_RSV_A.fasta \
    --rsv_b_sequences ${LIB_PATH}/sequences_RSV_B.fasta \
    --rsv_size_filter  \
    --deduplicate \
    > ${DB_PREP_DIR}/log 2>&1

rm: cannot remove '/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu//db_prep/refseq_ncbiFlu_kfv2_102424//kfdb.sqlite': No such file or directory


: 1

In [ ]:
tree ${DB_PREP_DIR}

## Prepare new kraken2 DB directory
Create the direcotry and copy the kraken-flu results into it, then use the kraken-build tool to add the library to the new DB

In [ ]:
rm -rf ${DB_PATH}
mkdir -p ${DB_PATH}

Copy the taxonomy created by kraken-flu in the new dir

In [ ]:
cp -r ${DB_PREP_DIR}/taxonomy ${DB_PATH}

also need the large NCBIU accession to tax ID file here for kraken-build

In [ ]:
ln -s ${TAX_PATH}/taxonomy/nucl_gb.accession2taxid ${DB_PATH}/taxonomy

In [ ]:
tree ${DB_PATH}

use kraken-tool to add the library file to the new DB

In [ ]:
mkdir ${DB_PATH}/library

Run the new library through kraken2-build "add to library" process.
This is still required because this is the step that does the masking, which is important for the "build" step of kraken2-build (next)

In [ ]:
kraken2-build \
    --add-to-library ${DB_PREP_DIR}/library/library.fna \
    --db ${DB_PATH}

## Create the kraken2 DB

In [ ]:
kraken2-build \
    --build \
    --db ${DB_PATH}

In [ ]:
echo ${DB_PATH}

## Test with simulated data
Run with a set of simulated reads for all the current viruses of interest.


In [ ]:
export TEST_BASE_DIR=/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/krakenDBs/kraken_flu_test_kfv2_102424

In [ ]:
mkdir -p ${TEST_BASE_DIR}

In [ ]:
echo ${TEST_BASE_DIR}

In [ ]:
kraken2 \
    --db ${DB_PATH} \
    --output ${TEST_BASE_DIR}/output.kraken \
    --paired \
    --classified-out ${TEST_BASE_DIR}/class_seqs#.fq \
    --unclassified-out ${TEST_BASE_DIR}/unclass_seqs#.fq \
    --report ${TEST_BASE_DIR}/report.txt \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-1.fq \
/lustre/scratch126/gsu/team112/personal/fs5/rvi_dev/sim_reads/art/mix/mix_of_all_01-2.fq

In [ ]:
cat ${TEST_BASE_DIR}/report.txt 